In [1]:
import pandas as pd
from transformers import AutoTokenizer

c:\Users\sebas\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
bert_model_name = "bert-base-multilingual-cased"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

In [3]:
train = pd.read_parquet("data/train-00000-of-00001.parquet")
dev = pd.read_parquet("data/dev-00000-of-00001.parquet")
test = pd.read_parquet("data/test-00000-of-00001.parquet")

In [4]:
train.head()

,text,ents,sents,tokens,spans,dagw_source,dagw_domain,dagw_source_full
0,Danmark skal bygges af maskinernes forsigtige ...,"[{'start': 0, 'end': 7, 'label': 'GPE'}]","[{'start': 0, 'end': 60}]","[{'id': 0, 'start': 0, 'end': 7}, {'id': 1, 's...",{'incorrect_spans': []},danavis,News,Danish daily newspapers
1,Hvil i Cap Dag - 2011 år Cap D'Agde Privat bil...,"[{'start': 17, 'end': 21, 'label': 'DATE'}, {'...","[{'start': 0, 'end': 79}]","[{'id': 0, 'start': 0, 'end': 4}, {'id': 1, 's...",{'incorrect_spans': []},cc,Web,Common Crawl
2,Måtte det nye år gøre dem mere fortrolige med ...,"[{'start': 6, 'end': 16, 'label': 'DATE'}, {'s...","[{'start': 0, 'end': 171}]","[{'id': 0, 'start': 0, 'end': 5}, {'id': 1, 's...",{'incorrect_spans': []},naat,Conversation,NAAT
3,Vi har også 360 graders Private Banking rådgiv...,"[{'start': 12, 'end': 23, 'label': 'QUANTITY'}]","[{'start': 0, 'end': 117}]","[{'id': 0, 'start': 0, 'end': 2}, {'id': 1, 's...",{'incorrect_spans': []},cc,Web,Common Crawl
4,| Nyhedsmails,[],"[{'start': 0, 'end': 13}]","[{'id': 0, 'start': 0, 'end': 1}, {'id': 1, 's...",{'incorrect_spans': []},cc,Web,Common Crawl


In [28]:
line = train.iloc[1]
text = line['text']
ents = line['ents']
tokens = line['tokens']
ents_list = []
for token in tokens:
    start = token['start']
    end = token['end']
    if len(ents) > 0:
        ent = ents[0]
        ent_start = ent['start']
        ent_end = ent['end']
        ent_label = ent['label']
        if end == ent_end:
            ents_list.append(ent_label)
            ents = ents[1:]
        elif start >= ent_start and end < ent_end:
            ents_list.append(ent_label)
        else:
            ents_list.append('O')
    else:
        ents_list.append('O')
print(ents_list)
print(len(ents_list))

['O', 'O', 'O', 'O', 'O', 'DATE', 'O', 'GPE', 'GPE', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
16


In [33]:
def ents_to_list(line):
    ents = line['ents']
    tokens = line['tokens']
    ents_list = []

    for token in tokens:
        start = token['start']
        end = token['end']

        if len(ents) > 0:
            ent = ents[0]
            ent_start = ent['start']
            ent_end = ent['end']
            ent_label = ent['label']

            if end == ent_end:
                ents_list.append(ent_label)
                ents = ents[1:]
            elif start >= ent_start and end < ent_end:
                ents_list.append(ent_label)
            else:
                ents_list.append('O')
                
        else:
            ents_list.append('O')
    
    return ents_list

In [35]:
test_idx = 2
line = train.iloc[test_idx]
ents_to_list(line)

Måtte det nye år gøre dem mere fortrolige med livet i Danmark og hjælpe dem, de ældre såvel som den opvoksende generation, til at finde sig til rette i det danske samfund.
[{'start': 6, 'end': 16, 'label': 'DATE'}
 {'start': 54, 'end': 61, 'label': 'GPE'}
 {'start': 156, 'end': 162, 'label': 'NORP'}]
[{'id': 0, 'start': 0, 'end': 5} {'id': 1, 'start': 6, 'end': 9}
 {'id': 2, 'start': 10, 'end': 13} {'id': 3, 'start': 14, 'end': 16}
 {'id': 4, 'start': 17, 'end': 21} {'id': 5, 'start': 22, 'end': 25}
 {'id': 6, 'start': 26, 'end': 30} {'id': 7, 'start': 31, 'end': 41}
 {'id': 8, 'start': 42, 'end': 45} {'id': 9, 'start': 46, 'end': 51}
 {'id': 10, 'start': 52, 'end': 53} {'id': 11, 'start': 54, 'end': 61}
 {'id': 12, 'start': 62, 'end': 64} {'id': 13, 'start': 65, 'end': 71}
 {'id': 14, 'start': 72, 'end': 75} {'id': 15, 'start': 75, 'end': 76}
 {'id': 16, 'start': 77, 'end': 79} {'id': 17, 'start': 80, 'end': 85}
 {'id': 18, 'start': 86, 'end': 91} {'id': 19, 'start': 92, 'end': 95}
 {

['O',
 'DATE',
 'DATE',
 'DATE',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'GPE',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'NORP',
 'O',
 'O']